In [ ]:
import sys, os
sys.path.append(os.path.join(os.pardir, 'python'))

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as pl
import math
import pyvista
import ufl
import basix
import dolfinx as df
import dolfinx.fem.petsc
import utils
import sys, os
import pathlib
from mpi4py import MPI
from dolfinx import fem

In [ ]:
def t_0(x):
    return 1.-x[1] + 0.2*np.cos(x[0]*np.pi)*np.sin(x[1]*np.pi)

In [ ]:
# code for Stokes Equation
ne = 40
p = 1

# define the domain (unit square) of length and width ne, running in parallel
domain = df.mesh.create_unit_square(MPI.COMM_WORLD, ne, ne)

# define extra variables
Ra = fem.Constant(domain, df.default_scalar_type(10000.0))
gravity = fem.Constant(domain, df.default_scalar_type((0.0,-1.0)))

# define velocity and pressure element (and temperature element)
v_e = basix.ufl.element("Lagrange", domain.basix_cell(), p+1, shape=(domain.geometry.dim,))
p_e = basix.ufl.element("Lagrange", domain.basix_cell(), p)
t_e = basix.ufl.element("Lagrange", domain.basix_cell(), p)

# define mixed element
vPe = basix.ufl.mixed_element([v_e, p_e])

# define function spaces (mixed element and temperature)
V = fem.functionspace(domain, vPe)
VT = fem.functionspace(domain, t_e)
T_i = fem.Function(VT)
T_it = fem.Function(VT)  # temporary solution at each iteration
T_i.interpolate(t_0)
vp_i = fem.Function(V)
vp_it = fem.Function(V)  # temporary solution at each iteration
v_i = vp_i.sub(0)

# define eta (1 for isoviscous)
eta = ufl.exp(-6.907755278982137*T_i)
#eta = 1

# define Velocity and Pressure function space
V_v, _ = V.sub(0).collapse()
V_vx, _ = V_v.sub(0).collapse()
V_vy, _ = V_v.sub(1).collapse()
V_p, _ = V.sub(1).collapse()

# define test functions
v_t, p_t = ufl.TestFunctions(V)
t_t = ufl.TestFunction(VT)

# define trial functions
trial_v, trial_p = ufl.TrialFunctions(V)
trial_t = ufl.TrialFunction(VT)

# define boundary conditions
# x = 0 or x = 1 (left and right boundaries)
def boundary_D_1(x):
    return np.logical_or(np.isclose(x[0], 0), np.isclose(x[0], 1))

dofs_D_1 = fem.locate_dofs_geometrical((V.sub(0).sub(0), V_vx), boundary_D_1)
u_bc1 = fem.Function(V_vx)
u_bc1.x.array[:] = 0
bc1 = fem.dirichletbc(u_bc1, dofs_D_1, V.sub(0).sub(0))

# y = 0 or y = 1 (top and bottom boundaries)
def boundary_D_2(x):
    return np.logical_or(np.isclose(x[1], 0), np.isclose(x[1], 1))

dofs_D_2 = fem.locate_dofs_geometrical((V.sub(0).sub(1), V_vy), boundary_D_2)
u_bc2 = fem.Function(V_vy)
u_bc2.x.array[:] = 0
bc2 = fem.dirichletbc(u_bc2, dofs_D_2, V.sub(0).sub(1))

# p = 0 (Pressure at origin)
def boundary_D_3(x):
    return np.logical_and(np.isclose(x[0], 0), np.isclose(x[1], 0))

dofs_D_3 = fem.locate_dofs_geometrical((V.sub(1), V_p), boundary_D_3)
u_bc3 = fem.Function(V_p)
u_bc3.x.array[:] = 0
bc3 = fem.dirichletbc(u_bc3, dofs_D_3, V.sub(1))

# assemble weak form
K = ufl.inner(ufl.sym(ufl.grad(v_t)), 2*eta*ufl.sym(ufl.grad(trial_v)))*ufl.dx
G = -ufl.div(v_t)*trial_p*ufl.dx
D = -p_t*ufl.div(trial_v)*ufl.dx
S = K + G + D

L = -ufl.inner(v_t, gravity)*Ra*T_i*ufl.dx

# assemble problem solver and solve
problem = df.fem.petsc.LinearProblem(S, L, bcs=[bc1, bc2, bc3], u=vp_it, petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
#vp_i = problem.solve()

In [ ]:
# code for Heat equation

# define temperature boundary conditions
def boundary_D_4(x):
    return np.isclose(x[1], 1)

dofs_D_4 = fem.locate_dofs_geometrical(VT, boundary_D_4)
u_bc4 = fem.Function(VT)
u_bc4.x.array[:] = 0
bc4 = fem.dirichletbc(u_bc4, dofs_D_4)

def boundary_D_5(x):
    return np.isclose(x[1], 0)
    
dofs_D_5 = fem.locate_dofs_geometrical(VT, boundary_D_5)
u_bc5 = fem.Function(VT)
u_bc5.x.array[:] = 1
bc5 = fem.dirichletbc(u_bc5, dofs_D_5)

# assemble weak form
ST = (t_t*ufl.inner(v_i, ufl.grad(trial_t)) + ufl.inner(ufl.grad(t_t), ufl.grad(trial_t)))*ufl.dx

zeroes = fem.Constant(domain, df.default_scalar_type(0.0))
LT = zeroes*t_t*ufl.dx

problemT = df.fem.petsc.LinearProblem(ST, LT, bcs=[bc4, bc5], u=T_it, petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
#T_i = problemT.solve()

In [ ]:
def error():
    r = ufl.action(S, vp_i) - L
    rT = ufl.action(ST, T_i) - LT

    rvec = df.fem.assemble_vector(dolfinx.fem.form(r))
    rTvec = df.fem.assemble_vector(dolfinx.fem.form(rT))

    df.fem.set_bc(rvec.array, [bc1, bc2, bc3], scale = 0)
    df.fem.set_bc(rTvec.array, [bc4, bc5], scale = 0)

    R = rvec.petsc_vec.norm()
    RT = rTvec.petsc_vec.norm()

    mixed_R = (R**2 + RT**2)**(1/2)

    return mixed_R

In [ ]:
def v_rms():
    vC = v_i.collapse()
    vrms = fem.assemble_scalar(fem.form((ufl.dot(vC, vC)*ufl.dx)))
    vrms = np.sqrt(MPI.COMM_WORLD.allreduce(vrms, op=MPI.SUM))
    
    return vrms

In [ ]:
def top(x):
    return np.isclose(x[1], 1)
fdim = domain.topology.dim - 1
top_facets = df.mesh.locate_entities_boundary(domain, fdim, top)
top_values = np.full_like(top_facets, 1)
sorted_facets = np.argsort(top_facets)
facet_tag = df.mesh.meshtags(domain, fdim, top_facets[sorted_facets], top_values[sorted_facets])
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag)

def Nu():
    nu = -fem.assemble_scalar(fem.form(T_i.dx(1)*ds(1)))
    return MPI.COMM_WORLD.allreduce(nu, op=MPI.SUM)

In [ ]:
# loop for iteration
maxits = 50
mixed_R = 1
tol = 0.000001
alpha = 0.8
n = 0
while mixed_R > tol:
    if n > maxits: break
    vp_it = problem.solve()
    vp_i.x.array[:] = (1-alpha)*vp_i.x.array + alpha*vp_it.x.array
    T_it = problemT.solve()
    T_i.x.array[:] = (1-alpha)*T_i.x.array + alpha*T_it.x.array
    print(n)
    mixed_R = error()
    print(mixed_R)
    n+=1

In [ ]:
# visualize
utils.plot_vector(v_i.collapse(), glyph_factor=0.00009)
utils.plot_scalar(T_i)

In [ ]:
v_rms(), Nu()